### Import Data and Index

In [2]:
import bz2

data_path = 'idwiki-latest-pages-articles-multistream.xml.bz2'
index_path = 'idwiki-latest-pages-articles-multistream-index.txt.bz2'

### Index file contains the title of each article
(https://stackoverflow.com/questions/29020732/how-to-use-information-provided-in-wiki-downloads-index-file)

In [3]:
count_title_from_index = 0

for i, line in enumerate(bz2.BZ2File(index_path, 'r')):
    count_title_from_index += 1

count_title_from_index

1660717

In [4]:
count_title_from_data = 0

for i, line in enumerate(bz2.BZ2File(data_path, 'r')):
    line = line.decode('utf-8')

    if '<title>' in line:
        count_title_from_data += 1

count_title_from_data

KeyboardInterrupt: 

### Searching for related categories
Since many categories that have been listed doesn't have its own 'Wiki page' (e.g. Kategori: Emigran Taiwan di Amerika Serikat), I decided to list categories that have their own page, so we can retrieve the articles that belongs to each category.

We have the list of every article's title in index.txt, so we can use it to decide which category (that have been listed or maybe have not listed in the Excel) that have their own page.

Why we should only use the categories that have their own page?
For the reason that I use 'Category Approach' in this wikidata, and the information about what articles that belong to a category only available in the 'category page', we should only use the categories that have their own page.

The categories will be searched on the xml dump file, since we want to retrieve the content either from pywikibot or mwparserfromhell library

In [ ]:
list_category = []

for i, line in enumerate(bz2.BZ2File(data_path, 'r')):
    line = line.decode('utf-8')

    if '<title>Kategori:' in line:
        # remove <title></title> tag
        stripped_line = line[20:-9]
        list_category.append(stripped_line)

list_category

#### DB Connection

In [24]:
import MySQLdb
import dotenv

config = dotenv.dotenv_values('.env')

db_credentials = {
    'host' : 'localhost',
    'user' : config['USERNAME'],
    'password' : config['PASSWORD'],
    'database' : config['DATABASE_NAME']
}

conn = MySQLdb.connect(**db_credentials)
conn.close()

#### Selecting categories and its subcategories that will be used
Documentation: 
- https://www.mediawiki.org/wiki/Manual:Page_table
- https://www.mediawiki.org/wiki/Manual:Categorylinks_table

In [25]:
category_to_be_used = {}
category_without_subcategory = []
page_to_be_used = {}

In [26]:
def get_subcategories(conn, category):
    """
    :param conn: MySQL's connection: 
    :param category: Pages will be searched based on this category (case-sensitive):
    :return:
    
    Get category's subcategories that will be used.
    
    The category itself will not be appended to the dict, cl_to can be used to get the 'super categories'; unless it doesn't have subcategories.
    """
    
    # eliminates the separation using space between words in category
    if len(category.split()) > 1 :
        category = category.replace(" ", "_")
    
    cur = conn.cursor()
    cur.execute( f"select page_id, page_title, page_namespace, cl_to from page p inner join categorylinks c on p.page_id = c.cl_from "
                 f"where page_namespace = 14 and cl_to = '{category}'" )
    result = cur.fetchall()
    
    if (len(result)) == 0:
        category = category.replace("_", " ")
        if category not in category_without_subcategory:
            category_without_subcategory.append(category)
            print(f"Category without subcategory: {category}")
    else : 
        for i, line in enumerate(result):
            page_id = line[0]
            page_title = line[1].decode('utf-8').replace("_", " ")
            page_namespace = line[2]
            cl_to = line[3].decode('utf-8').replace("_", " ")
            
            # check if the key already exists; if so, only append the cl_to
            if page_id in category_to_be_used:
                # check duplicates in cl_to
                if cl_to not in category_to_be_used[page_id]['cl_to']:
                    category_to_be_used[page_id]['cl_to'].append(cl_to)
            else:
                #  add the new category where page_id as the key
                category_to_be_used[page_id] = {
                    "page_id": page_id,
                    "page_title" : page_title,
                    "page_namespace" : page_namespace,
                    "cl_to" : [cl_to]
                }

In [27]:
def get_page_from_category(conn, category):
    """
    :param conn: MySQL's connection: 
    :param category: Pages will be searched based on this category (case-sensitive):
    :return:
   
    Get all pages that belong to a category.
    """

    # eliminates the separation using space between words in category
    if len(category.split()) > 1 :
        category = category.replace(" ", "_")
    
    cur = conn.cursor()
    cur.execute( f"select page_id, page_title, page_namespace from page p inner join categorylinks c on p.page_id = c.cl_from "
                 f"where page_namespace = 0 and cl_to = '{category}'" )
    
    result = cur.fetchall()
    _compose_page_result(result)

def get_page_from_page_title(conn, page_title):
    """
    :param conn: MySQL's connection: 
    :param page_title: Page's title that will be searched (case-sensitive): 
    :return:

    Get a page by its title.
    """

    # eliminates the separation using space between words in page_title
    if len(page_title.split()) > 1 :
        page_title = page_title.replace(" ", "_")

    cur = conn.cursor()
    cur.execute( f"select page_id, page_title, page_namespace from page p where page_namespace = 0 and page_title = '{page_title}'")
    
    result = cur.fetchall()
    _compose_page_result(result)
            
def _compose_page_result(result):
    """
    :param result: SQL Fetch result: 
    :return: 
  
    Append page_to_be_used dictionary that will be used by get_page_from_category() and get_page_from_page_title().
    """
    for i, line in enumerate(result):
        page_id = line[0]
        page_title = line[1].decode('utf-8').replace("_", " ")
        page_namespace = line[2]

        # check if the key already exists
        if page_id not in page_to_be_used:
            page_to_be_used[page_id] = {
                "page_id": page_id,
                "page_title" : page_title,
                "page_namespace" : page_namespace,
            }
    

In [29]:
conn = MySQLdb.connect(**db_credentials)


get_subcategories(conn, "Sejarah Indonesia")
get_subcategories(conn, "Orde Lama")
get_subcategories(conn, "Republik Indonesia Serikat")
get_subcategories(conn, "Tokoh Orde Lama")
get_subcategories(conn, "Indonesia dalam tahun 1945")
get_subcategories(conn, "Indonesia dalam tahun 1946")
get_subcategories(conn, "Indonesia dalam tahun 1947")
get_subcategories(conn, "Indonesia dalam tahun 1948")
get_subcategories(conn, "Indonesia dalam tahun 1949")
get_subcategories(conn, "Indonesia dalam tahun 1950")
get_subcategories(conn, "Indonesia dalam tahun 1951")
get_subcategories(conn, "Indonesia dalam tahun 1952")
get_subcategories(conn, "Indonesia dalam tahun 1953")
get_subcategories(conn, "Indonesia dalam tahun 1954")
get_subcategories(conn, "Indonesia dalam tahun 1955")
get_subcategories(conn, "Indonesia dalam tahun 1956")
get_subcategories(conn, "Indonesia dalam tahun 1957")
get_subcategories(conn, "Indonesia dalam tahun 1958")
get_subcategories(conn, "Indonesia dalam tahun 1959")
get_subcategories(conn, "Indonesia dalam tahun 1960")
get_subcategories(conn, "Indonesia dalam tahun 1961")
get_subcategories(conn, "Indonesia dalam tahun 1962")
get_subcategories(conn, "Indonesia dalam tahun 1963")
get_subcategories(conn, "Indonesia dalam tahun 1964")
get_subcategories(conn, "Indonesia dalam tahun 1965")
get_subcategories(conn, "Proklamasi Kemerdekaan Indonesia")
get_subcategories(conn, "Perang Kemerdekaan Indonesia")


for key, value in category_to_be_used.items():
    get_page_from_category(conn, category_to_be_used[key]['page_title'])

for i, line in enumerate(category_without_subcategory):
    get_page_from_category(conn, line)

conn.close()

Category without subcategory: Orde Lama
Category without subcategory: Republik Indonesia Serikat


In [23]:
len(page_to_be_used)

1984

### mwparserfromhell and pywikibot library

In [42]:
import mwparserfromhell
import pywikibot
page_with_infobox = {}
page_without_infobox = {}

In [43]:
def get_infobox(page_title):
    """
    :param page_title: 
    :return: 
    
    Get infobox based on page title. 
    
    If the page doesn't have infobox, it will be appended to page_without_infobox (vice versa).
    """
    # eliminates the separation using space between words in page_title
    global information
    
    if len(page_title.split()) > 1 :
        page_title = page_title.replace(" ", "_")
        
    site = pywikibot.Site('id', 'wikipedia')
    page = pywikibot.Page(site, page_title)
    
    try :
        text = page.get()
        wikitext = mwparserfromhell.parse(text)
        infoboxes = wikitext.filter_templates(matches = 'Infobox')
        if infoboxes:
            for infobox in infoboxes :
                information = {param.name.strip_code().strip(): param.value.strip_code().strip() for param in infobox.params}
                # save the infobox's name as the first 'index' 
                information = {'infobox_name' : str(infobox.name).replace('\n', ''), **information}                
                
                # Append to the list of values
                if page_title.replace("_", " ") in page_with_infobox:
                        page_with_infobox[page_title.replace("_", " ")].append(information)
                # Make the values as list since the page can have multiple infoboxes
                else :
                    page_with_infobox[page_title.replace("_", " ")] = [information]
        else:
            page_without_infobox[page_title.replace("_", " ")] = wikitext
    except pywikibot.page._basepage.IsRedirectPageError as e :
        print(f"!!! Title {page_title} has {e} Exception !!!")

In [44]:
for key, value in page_to_be_used.items():
    get_infobox(page_to_be_used[key]['page_title'])

!!! Title Pangeran_Wira_Kasoema has Page [[id:Pangeran Wira Kasoema]] is a redirect page. Exception !!!
!!! Title Kalimantan_Tenggara has Page [[id:Kalimantan Tenggara]] is a redirect page. Exception !!!
!!! Title Temenggung_Setia_Pahlawan has Page [[id:Temenggung Setia Pahlawan]] is a redirect page. Exception !!!
!!! Title Sejarah_Cipta_Pulau_Jawa has Page [[id:Sejarah Cipta Pulau Jawa]] is a redirect page. Exception !!!
!!! Title Sejarah_pulau_Jawa has Page [[id:Sejarah pulau Jawa]] is a redirect page. Exception !!!
!!! Title PPKI has Page [[id:PPKI]] is a redirect page. Exception !!!
!!! Title Nederlands-Indische_gulden has Page [[id:Nederlands-Indische gulden]] is a redirect page. Exception !!!
!!! Title Nederlands-Indische_Tramweg_Maatschappij has Page [[id:Nederlands-Indische Tramweg Maatschappij]] is a redirect page. Exception !!!
!!! Title Hasnan_A._Habib has Page [[id:Hasnan A. Habib]] is a redirect page. Exception !!!
!!! Title A._A._Rifai has Page [[id:A. A. Rifai]] is a red

In [45]:
# Solving the redirect page
conn = MySQLdb.connect(**db_credentials)

get_page_from_page_title(conn, 'Panitia Persiapan Kemerdekaan Indonesia')
get_page_from_page_title(conn, 'Hasnan Habib')
get_page_from_page_title(conn, 'Achmad Rifai Manggabarani')

get_infobox('Panitia Persiapan Kemerdekaan Indonesia')
get_infobox('Hasnan Habib')
get_infobox('Achmad Rifai Manggabarani')

conn.close()

#### Selecting the 'Orde Lama' Article

In [46]:
import re
page_with_infobox_orla = {}
page_with_infobox_besides_orla = {}

for key, value in page_with_infobox.items():    
    is_orla = False
    for index_value, value_inside in enumerate(value):
        for key_final, value_final in value_inside.items():
            numbers = re.findall(r'\b\d+\b', page_with_infobox[key][index_value][key_final])
            if any(1945 <= int(num) <= 1965 for num in numbers):
                is_orla = True
                break
        break
    if is_orla:
        page_with_infobox_orla[key] = value
    else :
        page_with_infobox_besides_orla[key] = value

### What is the most infobox used in 'Orde Lama' articles

In [98]:
infobox_per_names = {}

for key in page_with_infobox_orla:
    for index, key_inside in enumerate(page_with_infobox_orla[key]):
        infobox_name = str(page_with_infobox_orla[key][index]['infobox_name'])
        value_final = page_with_infobox_orla[key][index].update({'page_title' : key})
        if infobox_name in infobox_per_names:
            infobox_per_names[infobox_name].append(page_with_infobox_orla[key][index])
        else:
            infobox_per_names[infobox_name] = [page_with_infobox_orla[key][index]]

In [99]:
infobox_per_names

{'Infobox Penulis': [{'infobox_name': 'Infobox Penulis',
   'name': 'Sobron Aidit',
   'image': 'Sobron Aidit, Pekan Buku Indonesia 1954, p190.jpg',
   'imagesize': '200px',
   'caption': 'Sobron Aidit tahun 1954',
   'pseudonym': 'Simon',
   'birthname': '',
   'birth_date': '',
   'birth_place': 'Tanjung Pandan, Belitung, Hindia Belanda',
   'death_date': '',
   'death_place': 'Paris, Prancis',
   'occupation': 'Sastrawan, guru',
   'nationality': 'Prancis',
   'ethnicity': 'Minangkabau, Melayu Indonesia',
   'religion': 'Katolik',
   'citizenship': '',
   'period': '',
   'genre': '',
   'subject': '',
   'movement': '',
   'notableworks': '',
   'spouse': '',
   'partner': '',
   'children': 'Wanita Tekun Pertiwi',
   'relatives': 'D.N. Aidit (kakak)Asahan Alham (adik)',
   'influences': '',
   'influenced': '',
   'awards': '',
   'signature': '',
   'website': '',
   'portaldisp': '',
   'page_title': 'Sobron Aidit'},
  {'infobox_name': 'Infobox Penulis',
   'name': 'Abdoel Moeis

In [100]:
most_used_infobox_orla = max(infobox_per_names, key=lambda k: len(infobox_per_names[k]))
most_used_infobox_orla

'Infobox Officeholder'

#### Get Categories from each article

In [51]:
def get_categories_from_page_title(conn, page_title, category_dict) :
    """
   :param conn: MySQL's connection: 
   :param page_title: An article's title:
   :param category_dict: Dictionary-based type for holding the categories of a page:
   :return:
  
   Get all categories that belong to a page.
   """

    # eliminates the separation using space between words in category
    if len(page_title.split()) > 1 :
        page_title = page_title.replace(" ", "_")

    cur = conn.cursor()
    cur.execute(f"""
                SELECT
                    GROUP_CONCAT(REPLACE(cl.cl_to, '_', ' ')) AS categories
                FROM
                page AS p LEFT JOIN categorylinks AS cl ON p.page_id = cl.cl_from
                WHERE
                p.page_namespace = 0 and p.page_title = "{page_title}"
                """)

    result = cur.fetchall()
    try :
        category_dict[page_title.replace("_", " ")] = result[0][0].decode('utf-8')
    except AttributeError  as e:
        if page_title == 'Achmad_Rifai_Manggabarani':
            category_dict[page_title.replace("_", " ")] = 'Kelahiran 1924, Kematian 2001, Meninggal usia 77, Pejuang kemerdekaan Indonesia, Tokoh Bugis, Tokoh Sulawesi Selatan, Tokoh dari Polewali Mandar, Tokoh Angkatan 45, Politikus Indonesia, Gubernur Sulawesi Selatan, Penerima Bintang Gerilya'
        else : 
            print(e)

In [52]:
conn = MySQLdb.connect(**db_credentials)

categories_from_page = {}
categories_from_page_orla = {}

# For all articles with infobox
for key, value in page_with_infobox.items():
    get_categories_from_page_title(conn, key, categories_from_page)
    
# For all 'Orde Lama' articles with infobox
for key, value in page_with_infobox_orla.items():
    get_categories_from_page_title(conn, key, categories_from_page_orla)
    
conn.close()

### Extract data to excel

In [54]:
import pandas as pd

df = pd.DataFrame(list(page_with_infobox.items()), columns=['Title', 'Infobox'])
df_cat = pd.DataFrame(list(categories_from_page.items()), columns=['Title', 'Categories'])
result_df = pd.merge(df_cat, df, on='Title')
result_df.to_excel('infobox.xlsx', index=False)

In [183]:
df = pd.DataFrame(list(page_without_infobox.items()), columns=['Title', 'Text'])
df.to_excel('noinfobox.xlsx', index=False)

##### Extract data to excel for 'Orde Lama' by its infobox name

In [159]:
from openpyxl import Workbook

dataframes = []
dataframes_keys = []
for key, values in infobox_per_names.items():
    dataframes.append(pd.DataFrame(values))
    dataframes_keys.append(key)

workbook = Workbook()
for i, df in enumerate(dataframes):
    # Create a new sheet with the specified name
    sheet = workbook.create_sheet(title=dataframes_keys[i][8:39])

    # Write column headers
    headers = list(df.columns)
    sheet.append(headers)

    # Write data rows
    for index, row in df.iterrows():
        sheet.append(row.tolist())

    # Adjust column widths based on content
    for column in sheet.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = min(max_length + 2, 30)
        sheet.column_dimensions[column[0].column_letter].width = adjusted_width

workbook.remove(workbook.active)
workbook.save('infobox-orla.xlsx')


#### Extract excel for RDF

In [161]:
# convert the file that want to be extracted to csv

import openpyxl
import csv

workbook = openpyxl.load_workbook('infobox-orla.xlsx')
sheet = workbook['military conflict']

with open('military-conflict.csv', 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)

    for row in sheet.iter_rows(min_row=1, values_only=True):
        csv_writer.writerow(row)

workbook.close()

In [166]:
import csv
import openpyxl
from rdflib import Graph, Literal, Namespace, URIRef

workbook = openpyxl.load_workbook('infobox-orla.xlsx')
sheet = workbook['military conflict']

#When left blank all entities will be of type owl:thing or add type relations when a csv header is "http://www.w3.org/1999/02/22-rdf-syntax-ns#type" the subjects will then be assigned the values in this column as object
subjectsClassName = "Roman_wars"
#------------------------------------------------------------------------------------------------------------------


prefix = ("http://example.com/kad2020/")
# make a graph
g = Graph()
owl = Namespace("http://www.w3.org/2002/07/owl#")
g.bind("owl", owl)
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
g.bind("rdf", rdf)
ex = Namespace(prefix)
g.bind("ex", ex)

rowIndex = 0
for row in sheet:
    colIndex = 0
    subj = URIRef(row[0])
    #add class to all if wanted
    if subjectsClassName != "":
        g.add(  (subj, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), URIRef(prefix+subjectsClassName))  )
        g.add(  (URIRef(prefix+subjectsClassName), URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#subClassOf"),URIRef("http://www.w3.org/2002/07/owl#Thing"))  )
    else:
        g.add(  (subj, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), URIRef("http://www.w3.org/2002/07/owl#Thing"))  )

    #First row
    if rowIndex==0:
        colIndex2 = 0
        for colVal in row:
            if colIndex2 > 0:
                cellBelow = sheet[rowIndex+1][colIndex2]
                isObject = not cellBelow.__contains__(" ") and cellBelow.__contains__("dbpedia")
                name =prefix + colVal
                if isObject:
                    g.add(  (URIRef(name), URIRef("http://www.w3.org/2002/07/owl#subPropertyOf"), URIRef("http://www.w3.org/2002/07/owl#topObjectProperty"))  )
                    g.add(   (URIRef(name), URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#Property")))
                    g.add((URIRef(name), URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"),
                           URIRef("http://www.w3.org/2002/07/owl#ObjectProperty")))
                if not isObject:
                    g.add(  (URIRef(name), URIRef("http://www.w3.org/2002/07/owl#subPropertyOf"), URIRef("http://www.w3.org/2002/07/owl#topDataProperty"))  )
                    g.add(   (URIRef(name), URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#Property")))
                    g.add((URIRef(name), URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"),
                           URIRef("http://www.w3.org/2002/07/owl#DatatypeProperty")))
            colIndex2+=1

    if rowIndex > 0:
        for colVal in row:
            if colIndex > 0:
                name = prefix + sheet[0][colIndex]
                pred = URIRef(name)
                if colVal != "":
                    if not colVal.__contains__(" ") and colVal.__contains__("dbpedia"):
                        obj = URIRef(colVal)
                    else:
                        obj = Literal(colVal)
                    g.add((subj, pred, obj))
            colIndex+=1
    rowIndex+=1

g.serialize(destination='example_output.ttl', format='turtle')

workbook.close()

TypeError: argument of type 'Cell' is not iterable